In [ ]:
import pandas as pd
import os

base_dir = "./privilage_analysis_data/data_cve/"

trace_parser = ["sub_sys", "sub_ip", "obj_ip", "cs"]

all_traces_cve_df = pd.DataFrame()

for idx, fileLocation in enumerate(os.listdir(base_dir)):

    filePath = os.path.join(base_dir, fileLocation, "trace.txt")
    if not os.path.isfile(filePath):
        continue
    print(filePath)
    
    df = pd.read_csv(filePath, names=trace_parser, skiprows=[0])
    df['trace_name'] = fileLocation
    df = df[(df['sub_sys']!=-1)]
    df = df[(df['obj_ip']!='eeeeffff')]
    df = df[(df['obj_ip']!='feedbeef')]
    df['tuple'] = 'ss' + df['sub_sys'].astype(str) + 'os' + df['obj_ip'].astype(str)

    #print(df.info())
    df['index'] = df.reset_index().index.to_list()
        
    df.dropna(inplace=True)
        
    df['float_obj_ip'] = df['obj_ip'].astype(str).apply(int, base=16)

    df['deltas'] = abs(df['float_obj_ip'] - df['float_obj_ip'].shift(1))
    
    df['deltas'] = df['deltas'].astype(str)
    
    df.dropna(inplace=True)

    all_traces_cve_df = pd.concat([all_traces_cve_df, df])

#all_traces_cve_df.reset_index(inplace=True)
print(all_traces_cve_df.shape)
print(all_traces_cve_df.info())
all_traces_cve_df = all_traces_cve_df[~all_traces_cve_df.obj_ip.isna()]

In [ ]:
import pandas as pd
import os

base_dir = "./privilage_analysis_data/data_all/"

trace_parser = ["sub_sys", "sub_ip", "obj_ip", "cs"]

all_traces_df = pd.DataFrame()

for idx, fileLocation in enumerate(os.listdir(base_dir)):

    filePath = os.path.join(base_dir, fileLocation, "trace.txt")
    if not os.path.isfile(filePath):
        continue
    print(filePath)
    
    df = pd.read_csv(filePath, names=trace_parser, skiprows=[0])
    df['trace_name'] = fileLocation
    #print(df.info())
    df = df[(df['sub_sys']!=-1)]
    df = df[(df['obj_ip']!='eeeeffff')]
    df = df[(df['obj_ip']!='feedbeef')]
    df['tuple'] = 'ss' + df['sub_sys'].astype(str) + 'os' + df['obj_ip'].astype(str)
    
    df['index'] = df.reset_index().index.to_list()
    
    df.dropna(inplace=True)
        
    df['float_obj_ip'] = df['obj_ip'].astype(str).apply(int, base=16)

    df['deltas'] = abs(df['float_obj_ip'] - df['float_obj_ip'].shift(1))
    
    df['deltas'] = df['deltas'].astype(str)
    
    df.dropna(inplace=True)

    all_traces_df = pd.concat([all_traces_df, df])

#all_traces_df.reset_index(inplace=True)
print(all_traces_df.shape)
print(all_traces_df.info())
all_traces_df = all_traces_df[~all_traces_df.obj_ip.isna()]

In [ ]:
# 1000,2000, 6000, 8000, 10000
sequence_length = 8000
test_size=0.20 
random_state=42
# 1, 20, 50, 80, 100, 200
sentence_length = 100
epochs=30

In [ ]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

from pprint import pprint  # pretty-printer
from collections import defaultdict

from sklearn.model_selection import train_test_split
from collections import defaultdict

## vectorize
all_df = pd.concat([all_traces_df,  all_traces_cve_df])
all_df.shape

vectorized_traces = pd.DataFrame()
vectorized_traces = all_df.groupby(['trace_name'], as_index=False).count()[['trace_name', 'index']]
sequenced_traces = all_df.groupby(['trace_name'], as_index=False)['obj_ip'].apply(lambda x: ' '.join(x))

vectorized_traces = pd.merge(
    sequenced_traces,
    vectorized_traces,
    how="inner",
    on='trace_name',
    sort=True,
)
vectorized_traces

vectorized_traces['trace_label'] = vectorized_traces['trace_name'].str.split('-').str[0]
vectorized_traces['label'] = vectorized_traces["trace_label"].astype('category').cat.codes
vectorized_traces['binary_label'] = np.where(vectorized_traces['trace_label'].str.contains('CVE'), 0, 1)

a, b = train_test_split(vectorized_traces[vectorized_traces['binary_label'] ==1], test_size=0.50, random_state=2, stratify=vectorized_traces[vectorized_traces['binary_label'] ==1]['trace_label']) 
at = vectorized_traces[vectorized_traces['binary_label'] ==0].copy()
vectorized_traces = pd.concat([at, a])
training_traces = vectorized_traces.iloc[51:]

testing_traces = vectorized_traces.iloc[0:50]
testing_traces
training_traces.groupby(['trace_label'], as_index=False).count()

#-----------

binary_labels = vectorized_traces['binary_label']
X_train, X_test, y_train, y_test, train_index, test_index = train_test_split(vectorized_traces, binary_labels, vectorized_traces.index.to_list(), test_size=0.20, random_state=42, stratify=binary_labels) 

df1, df2 = X_train[(mask:=(X_train['trace_name'].str.contains('CVE_2022_31440')) | (X_train['trace_name'].str.contains('hdparm')))].copy(), X_train[~mask].copy()

vectorized_traces[vectorized_traces['binary_label'] == 1]

tokenized_all_traces = [
    [word for word in trace.split()]
    for trace in vectorized_traces['obj_ip'].to_numpy()
]

tokenized_traces = [
    [word for word in trace.split()]
    for trace in training_traces['obj_ip'].to_numpy()
]

tokenized_train_traces = [
    [word for word in trace.split()]
    for trace in X_train['obj_ip'].to_numpy()
]

tokenized_test_traces = [
    [word for word in trace.split()]
    for trace in testing_traces['obj_ip'].to_numpy()
]

tokenized_test_model_traces = [
    [word for word in trace.split()]
    for trace in X_test['obj_ip'].to_numpy()
]

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

## tokenize text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_traces + tokenized_test_traces)
word_index = tokenizer.word_index
vocab_size = len(word_index)
vocab_size

## padding data
#sequence_length = 8000

all_sequences = tokenizer.texts_to_sequences(tokenized_all_traces)
padded_all_seq = pad_sequences(all_sequences, maxlen=sequence_length, padding='pre', truncating='post')

train_sequences = tokenizer.texts_to_sequences(tokenized_traces)
padded_train_seq = pad_sequences(train_sequences, maxlen=sequence_length, padding='pre', truncating='post')

test_sequences = tokenizer.texts_to_sequences(tokenized_test_traces)
padded_test_seq = pad_sequences(test_sequences, maxlen=sequence_length, padding='pre', truncating='post')

train_model_sequences = tokenizer.texts_to_sequences(tokenized_train_traces)
padded_train_model_seq = pad_sequences(train_model_sequences, maxlen=sequence_length, padding='pre', truncating='post')

test_model_sequences = tokenizer.texts_to_sequences(tokenized_test_model_traces)
padded_test_model_seq = pad_sequences(test_model_sequences, maxlen=sequence_length, padding='pre', truncating='post')

print(padded_train_model_seq.shape, padded_test_model_seq.shape)

## Whole Seq w=1

In [ ]:
from random import randint
from numpy import array
from numpy import argmax

from tensorflow.keras import models
from numpy import array_equal
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Input
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

#sentence_length = 1
sentence_num = padded_train_model_seq.shape[1]//sentence_length
batch_size = padded_train_model_seq.shape[0]
whole_seq_length = padded_train_model_seq.shape[1]

padded_train_model_seq_reshaped = np.reshape(padded_train_model_seq, (batch_size, sentence_num, sentence_length, 1))

#inputs = Input(shape=(100, 50, 1))
#lstm_sentence_layer = LSTM(128)
#sentence_inputs = TimeDistributed(lstm_sentence_layer)(inputs)
#sentence_inputs.shape

ms = Sequential()
ms.add(Input(shape=(whole_seq_length, 1)))
#ms.add(TimeDistributed(Bidirectional(LSTM(128))))
#ms.add(Dropout(0.2))
ms.add(Bidirectional(LSTM(128, return_sequences=True)))
ms.add(Dropout(0.2))
ms.add(LSTM(128))
ms.add(Dropout(0.2))
ms.add(Dense(1, activation='sigmoid'))


#m = Sequential([
#    #Input(tensor=(100, 50, 1)),
#    TimeDistributed(Bidirectional(LSTM(128))),
#    Dropout(0.2),
#    Bidirectional(LSTM(128, return_sequences=True, stateful=True)),
#    Dropout(0.2),
#    LSTM(128, stateful=True),
#    Dropout(0.2),
#    Dense(28, activation='softmax')
#    #,
#    #Dropout(0.2),
#    #Dense(256),
#    #Dense(1, activation='sigmoid')
#])


mod = Model()

ms.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')
ms.summary()

history = ms.fit(padded_train_model_seq_reshaped, y_train, epochs=epochs)

ms.save('w'+str(sentence_length)+'_l'+str(sequence_length)+'_no_max_pooling_no_embedding')

## Whole Seq w=1 with Embedding

In [ ]:
from random import randint
from numpy import array
from numpy import argmax

from tensorflow.keras import models
from numpy import array_equal
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Input
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout, Embedding

#sentence_length = 1
sentence_num = padded_train_model_seq.shape[1]//sentence_length
batch_size = padded_train_model_seq.shape[0]
whole_seq_length = padded_train_model_seq.shape[1]

padded_train_model_seq_reshaped = np.reshape(padded_train_model_seq, (batch_size, sentence_num, sentence_length, 1))

#inputs = Input(shape=(100, 50, 1))
#lstm_sentence_layer = LSTM(128)
#sentence_inputs = TimeDistributed(lstm_sentence_layer)(inputs)
#sentence_inputs.shape

ms = Sequential()
ms.add(Input(shape=(whole_seq_length, 1)))
ms.add(Embedding(vocab_size, 64))
#ms.add(TimeDistributed(Bidirectional(LSTM(128))))
#ms.add(Dropout(0.2))
ms.add(Bidirectional(LSTM(128, return_sequences=True)))
ms.add(Dropout(0.2))
ms.add(LSTM(128))
ms.add(Dropout(0.2))
ms.add(Dense(1, activation='sigmoid'))


#m = Sequential([
#    #Input(tensor=(100, 50, 1)),
#    TimeDistributed(Bidirectional(LSTM(128))),
#    Dropout(0.2),
#    Bidirectional(LSTM(128, return_sequences=True, stateful=True)),
#    Dropout(0.2),
#    LSTM(128, stateful=True),
#    Dropout(0.2),
#    Dense(28, activation='softmax')
#    #,
#    #Dropout(0.2),
#    #Dense(256),
#    #Dense(1, activation='sigmoid')
#])


mod = Model()

ms.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')
ms.summary()

history = ms.fit(padded_train_model_seq_reshaped, y_train, epochs=epochs)

ms.save('w'+str(sentence_length)+'_l'+str(sequence_length)+'_no_max_pooling_no_embedding')

## Sliced Seq w != 1

In [ ]:
from random import randint
from numpy import array
from numpy import argmax

from tensorflow.keras import models
from numpy import array_equal
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Input
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

#sentence_length = 100
sentence_num = padded_train_model_seq.shape[1]//sentence_length
batch_size = padded_train_model_seq.shape[0]
whole_seq_length = padded_train_model_seq.shape[1]

padded_train_model_seq_reshaped = np.reshape(padded_train_model_seq, (batch_size, sentence_num, sentence_length, 1))

#inputs = Input(shape=(100, 50, 1))
#lstm_sentence_layer = LSTM(128)
#sentence_inputs = TimeDistributed(lstm_sentence_layer)(inputs)
#sentence_inputs.shape
ms.reset_states()

ms = Sequential()
ms.add(Input(shape=(sentence_num, sentence_length, 1)))
ms.add(TimeDistributed(Bidirectional(LSTM(128))))
ms.add(Dropout(0.2))
ms.add(Bidirectional(LSTM(128, return_sequences=True)))
ms.add(Dropout(0.2))
ms.add(LSTM(128))
ms.add(Dropout(0.2))
ms.add(Dense(1, activation='sigmoid'))


#m = Sequential([
#    #Input(tensor=(100, 50, 1)),
#    TimeDistributed(Bidirectional(LSTM(128))),
#    Dropout(0.2),
#    Bidirectional(LSTM(128, return_sequences=True, stateful=True)),
#    Dropout(0.2),
#    LSTM(128, stateful=True),
#    Dropout(0.2),
#    Dense(28, activation='softmax')
#    #,
#    #Dropout(0.2),
#    #Dense(256),
#    #Dense(1, activation='sigmoid')
#])


mod = Model()

ms.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')
ms.summary()


history = ms.fit(padded_train_model_seq_reshaped, y_train, epochs=40)

#ms.save('w'+str(sentence_length)+'_l'+str(sequence_length)+'_no_max_pooling_no_embedding')

## Sliced Seq with Embedding 

In [ ]:
ms.evaluate(padded_test_model_seq_reshaped, y_test, verbose=1)

In [ ]:
from random import randint
from numpy import array
from numpy import argmax

from tensorflow.keras import models
from numpy import array_equal
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Input
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout, Embedding

#sentence_length = 100
sentence_num = padded_train_model_seq.shape[1]//80
batch_size = padded_train_model_seq.shape[0]
whole_seq_length = padded_train_model_seq.shape[1]

padded_train_model_seq_reshaped = np.reshape(padded_train_model_seq, (batch_size, sentence_num, 80, 1))

#inputs = Input(shape=(100, 50, 1))
#lstm_sentence_layer = LSTM(128)
#sentence_inputs = TimeDistributed(lstm_sentence_layer)(inputs)
#sentence_inputs.shape
embed_ms.reset_states()


embed_ms = Sequential()
embed_ms.add(Input(shape=(sentence_num, 80)))
embed_ms.add(Embedding(vocab_size, 64))
embed_ms.add(TimeDistributed(Bidirectional(LSTM(128))))
embed_ms.add(Dropout(0.2))
embed_ms.add(Bidirectional(LSTM(128, return_sequences=True)))
embed_ms.add(Dropout(0.2))
embed_ms.add(LSTM(128))
embed_ms.add(Dropout(0.2))
embed_ms.add(Dense(1, activation='sigmoid'))


#m = Sequential([
#    #Input(tensor=(100, 50, 1)),
#    TimeDistributed(Bidirectional(LSTM(128))),
#    Dropout(0.2),
#    Bidirectional(LSTM(128, return_sequences=True, stateful=True)),
#    Dropout(0.2),
#    LSTM(128, stateful=True),
#    Dropout(0.2),
#    Dense(28, activation='softmax')
#    #,
#    #Dropout(0.2),
#    #Dense(256),
#    #Dense(1, activation='sigmoid')
#])


mod = Model()

embed_ms.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')
embed_ms.summary()

history = embed_ms.fit(padded_train_model_seq_reshaped, y_train, epochs=40)

#embed_ms.save('w'+str(sentence_length)+'_l'+str(sequence_length)+'_no_max_pooling_with_embedding')

In [ ]:
saved_ms = keras.models.load_model('w1_l8000_no_max_pooling_no_embedding')

In [ ]:
from tensorflow import keras
import os
 
evaluation = pd.DataFrame(columns=['loss', 'accuracy', 'w', 'l'])
i=1
for x in os.listdir():
    if "no_max_pooling_no_embedding" in x:
        saved_ms = keras.models.load_model(x)
        l = x.split('_')
        window = int(l[0][1:])
        length = int(l[1][1:])
        padded_test_model_seq = pad_sequences(test_model_sequences, maxlen=length, padding='pre', truncating='post')
        padded_test_model_seq_reshaped = np.reshape(padded_test_model_seq, (padded_test_model_seq.shape[0], length//window, window, 1))
        score = saved_ms.evaluate(padded_test_model_seq_reshaped, y_test, verbose=1)
        evaluation.loc[len(evaluation)]=score + [window] + [length]
        print(i)
        print(x, window, length)
        i+=1








In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

padded_all_seq = pad_sequences(all_sequences, maxlen=8000, padding='pre', truncating='post')
 
documents = [','.join(x) for x in padded_all_seq.astype(str)]
# Create a Vectorizer Object
vectorizer = CountVectorizer()
 
vectorizer.fit(documents)

binary_labels = vectorized_traces['binary_label']
#X_train, X_test, y_train, y_test, train_index, test_index = train_test_split(documents, binary_labels, vectorized_traces.index.to_list(), test_size=0.20, random_state=42, stratify=binary_labels) 
 
# Printing the identified Unique words along with their indices
#print("Vocabulary: ", vectorizer.vocabulary_)
 
# Encode the Document
documents = [','.join(x) for x in padded_all_seq.astype(str)]

vector = vectorizer.transform(documents)

binary_labels = vectorized_traces['binary_label']
X_train_cf, X_test_cf, y_train_cf, y_test_cf, train_index_cf, test_index_cf = train_test_split(vector, binary_labels, vectorized_traces.index.to_list(), test_size=0.20, random_state=42, stratify=binary_labels) 
 


In [ ]:
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB

# MNB CLASSIFICATION
mnb = MultinomialNB()
mnb.fit(X_train_cf,y_train_cf)
mnb_prediction = mnb.predict(X_test_cf)

mnb_results = np.array(list(zip(y_test_cf,mnb_prediction)))
mnb_results = pd.DataFrame(mnb_results, columns=['id', 'result'])
mnb_results.to_csv('mnb_vectorized.csv', index = False)

print(classification_report(y_test_cf, mnb_prediction))

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix


acc_score = accuracy_score(y_test_cf,mnb_prediction)
conf_mat = confusion_matrix(y_test_cf, mnb_prediction)
print(acc_score)
print(conf_mat)

In [ ]:
evaluation.sort_values(['loss', 'accuracy'])

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from tensorflow import keras

ms = keras.models.load_model('w80_l8000_no_max_pooling_no_embedding')

#padded_test_model_seq = pad_sequences(test_model_sequences, maxlen=8000, padding='pre', truncating='post')
padded_test_model_seq_reshaped = np.reshape(padded_test_model_seq, (padded_test_model_seq.shape[0], 8000//80, 80, 1))

y_pred_lstm = ms.predict(padded_test_model_seq_reshaped)

lstm = pd.DataFrame()                                                             
lstm['pred'] = y_pred_lstm.ravel()
lstm['true'] = y_test.to_list()  

lstm_exploit = lstm[lstm['true']==0].copy()

lstm_exploit_1  = pd.DataFrame()
lstm_exploit_1['true'] = 1- lstm_exploit['true']
lstm_exploit_1['pred'] = 1- lstm_exploit['pred']
lstm_exploit = pd.concat([lstm_exploit,lstm_exploit_1]) 
                                                                   
fpr_lstm_0, tpr_lstm_0, threshold_lstm_0 = roc_curve(lstm_exploit['true'], lstm_exploit['pred'])
auc_lstm_0 = auc(fpr_lstm_0, tpr_lstm_0)

lstm_normal = lstm[lstm['true']==1].copy()

lstm_normal_0  = pd.DataFrame()
lstm_normal_0['true'] = 1- lstm_normal['true']
lstm_normal_0['pred'] = 1- lstm_normal['pred']
lstm_normal = pd.concat([lstm_normal,lstm_normal_0])
                                                                    
fpr_lstm_1, tpr_lstm_1, threshold_lstm_1 = roc_curve(lstm_normal['true'], lstm_normal['pred'])
auc_lstm_1 = auc(fpr_lstm_1, tpr_lstm_1) 
                                                                    
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_lstm_0, tpr_lstm_0, label='exploit (area = {:.4f})'.format(auc_lstm_0))
plt.plot(fpr_lstm_1, tpr_lstm_1, label='normal (area = {:.4f})'.format(auc_lstm_1))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('Two Stage LSTM without Embedding')
plt.legend(loc='best')
plt.show()

lstm_acc_score = accuracy_score(y_test, y_pred_lstm.ravel().round())
lstm_conf_mat = confusion_matrix(y_test, y_pred_lstm.ravel().round())
print(lstm_acc_score)
print(lstm_conf_mat)

# Supervised transformation based on random forests
rf = RandomForestClassifier(max_depth=3, n_estimators=10)
rf.fit(X_train_cf, y_train_cf)
#mnb = MultinomialNB()
#mnb.fit(X_train,y_train)
#mnb_prediction = mnb.predict(X_test)
y_pred_rf = rf.predict_proba(X_test_cf)[:, 1]

rf = pd.DataFrame()                                                             
rf['pred'] = y_pred_rf.ravel()
rf['true'] = y_test_cf.to_list()  

rf_exploit = rf[rf['true']==0].copy()

rf_exploit_1  = pd.DataFrame()
rf_exploit_1['true'] = 1- rf_exploit['true']
rf_exploit_1['pred'] = 1- rf_exploit['pred']
rf_exploit = pd.concat([rf_exploit,rf_exploit_1]) 
                                                                   
fpr_rf_0, tpr_rf_0, threshold_rf_0 = roc_curve(rf_exploit['true'], rf_exploit['pred'])
auc_rf_0 = auc(fpr_rf_0, tpr_rf_0)

rf_normal = rf[rf['true']==1].copy()

rf_normal_0  = pd.DataFrame()
rf_normal_0['true'] = 1- rf_normal['true']
rf_normal_0['pred'] = 1- rf_normal['pred']
rf_normal = pd.concat([rf_normal,rf_normal_0])
                                                                    
fpr_rf_1, tpr_rf_1, threshold_rf_1 = roc_curve(rf_normal['true'], rf_normal['pred'])
auc_rf_1 = auc(fpr_rf_1, tpr_rf_1) 
                                                                
plt.figure(2)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_rf_0, tpr_rf_0, label='exploit (area = {:.4f})'.format(auc_rf_0))
plt.plot(fpr_rf_1, tpr_rf_1, label='normal (area = {:.4f})'.format(auc_rf_1))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('Random Forest')
plt.legend(loc='best')
plt.show()
#fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test, mnb_prediction)
#auc_rf = auc(fpr_rf, tpr_rf)

rf_acc_score = accuracy_score(y_test_cf, y_pred_rf.ravel().round())
rf_conf_mat = confusion_matrix(y_test_cf, y_pred_rf.ravel().round())
print(rf_acc_score)
print(rf_conf_mat)


mnb = MultinomialNB()
mnb.fit(X_train_cf,y_train_cf)
mnb_prediction = mnb.predict(X_test_cf)
y_pred_mnb = mnb.predict_proba(X_test_cf)[:, 1]

mnb = pd.DataFrame()                                                             
mnb['pred'] = y_pred_mnb.ravel()
mnb['true'] = y_test_cf.to_list()  

mnb_exploit = mnb[mnb['true']==0].copy()

mnb_exploit_1  = pd.DataFrame()
mnb_exploit_1['true'] = 1- mnb_exploit['true']
mnb_exploit_1['pred'] = 1- mnb_exploit['pred']
mnb_exploit = pd.concat([mnb_exploit,mnb_exploit_1]) 
                                                                   
fpr_mnb_0, tpr_mnb_0, threshold_mnb_0 = roc_curve(mnb_exploit['true'], mnb_exploit['pred'])
auc_mnb_0 = auc(fpr_mnb_0, tpr_mnb_0)

mnb_normal = mnb[mnb['true']==1].copy()

mnb_normal_0  = pd.DataFrame()
mnb_normal_0['true'] = 1- mnb_normal['true']
mnb_normal_0['pred'] = 1- mnb_normal['pred']
mnb_normal = pd.concat([mnb_normal,mnb_normal_0])
                                                                    
fpr_mnb_1, tpr_mnb_1, threshold_mnb_1 = roc_curve(mnb_normal['true'], mnb_normal['pred'])
auc_mnb_1 = auc(fpr_mnb_1, tpr_mnb_1) 
                                                                
plt.figure(3)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_mnb_0, tpr_mnb_0, label='exploit (area = {:.4f})'.format(auc_mnb_0))
plt.plot(fpr_mnb_1, tpr_mnb_1, label='normal (area = {:.4f})'.format(auc_mnb_1))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('Naive Bayes')
plt.legend(loc='best')
plt.show()

mnb_acc_score = accuracy_score(y_test_cf, y_pred_mnb.ravel().round())
mnb_conf_mat = confusion_matrix(y_test_cf, y_pred_mnb.ravel().round())
print(mnb_acc_score)
print(mnb_conf_mat)

#saved_ms_lstm_embedded = keras.models.load_model('w100_l8000_no_max_pooling_with_embedding')

#padded_test_model_seq = pad_sequences(test_model_sequences, maxlen=8000, padding='pre', truncating='post')
#padded_test_model_seq_reshaped = np.reshape(padded_test_model_seq, (padded_test_model_seq.shape[0], 8000//100, 100, 1))

y_pred_lstm_embedded = embed_ms.predict(padded_test_model_seq_reshaped)

lstm_embedded = pd.DataFrame()                                                             
lstm_embedded['pred'] = y_pred_lstm_embedded.ravel()
lstm_embedded['true'] = y_test.to_list()  

lstm_embedded_exploit = lstm_embedded[lstm_embedded['true']==0].copy()

lstm_embedded_exploit_1  = pd.DataFrame()
lstm_embedded_exploit_1['true'] = 1- lstm_embedded_exploit['true']
lstm_embedded_exploit_1['pred'] = 1- lstm_embedded_exploit['pred']
lstm_embedded_exploit = pd.concat([lstm_embedded_exploit,lstm_embedded_exploit_1]) 
                                                                   
fpr_lstm_embedded_0, tpr_lstm_embedded_0, threshold_lstm_embedded_0 = roc_curve(lstm_embedded_exploit['true'], lstm_embedded_exploit['pred'])
auc_lstm_embedded_0 = auc(fpr_lstm_embedded_0, tpr_lstm_embedded_0)

lstm_embedded_normal = lstm_embedded[lstm_embedded['true']==1].copy()

lstm_embedded_normal_0  = pd.DataFrame()
lstm_embedded_normal_0['true'] = 1- lstm_embedded_normal['true']
lstm_embedded_normal_0['pred'] = 1- lstm_embedded_normal['pred']
lstm_embedded_normal = pd.concat([lstm_embedded_normal,lstm_embedded_normal_0])
                                                                    
fpr_lstm_embedded_1, tpr_lstm_embedded_1, threshold_lstm_embedded_1 = roc_curve(lstm_embedded_normal['true'], lstm_embedded_normal['pred'])
auc_lstm_embedded_1 = auc(fpr_lstm_embedded_1, tpr_lstm_embedded_1) 
                                                                    
plt.figure(4)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_lstm_embedded_0, tpr_lstm_embedded_0, label='exploit (area = {:.4f})'.format(auc_lstm_embedded_0))
plt.plot(fpr_lstm_embedded_1, tpr_lstm_embedded_1, label='normal (area = {:.4f})'.format(auc_lstm_embedded_1))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('Two Stage lstm_embedded with Embedding')
plt.legend(loc='best')
plt.show()

lstm_embedded_acc_score = accuracy_score(y_test, y_pred_lstm_embedded.ravel().round())
lstm_embedded_conf_mat = confusion_matrix(y_test, y_pred_lstm_embedded.ravel().round())
print(lstm_embedded_acc_score)
print(lstm_embedded_conf_mat)

In [ ]:
saved_ms = keras.models.load_model('w100_l8000_no_max_pooling_no_embedding')

padded_test_model_seq = pad_sequences(test_model_sequences, maxlen=8000, padding='pre', truncating='post')
padded_test_model_seq_reshaped = np.reshape(padded_test_model_seq, (padded_test_model_seq.shape[0], 8000//100, 100, 1))

y_pred_keras = saved_ms.predict(padded_test_model_seq_reshaped)

In [ ]:
lstm_exploit_1  = pd.DataFrame()
lstm_exploit = lstm[lstm['true']==0].copy()
lstm_exploit_1['true'] = 1- lstm_exploit['true']
lstm_exploit_1['pred'] = 1- lstm_exploit['pred']

lstm_normal = lstm[lstm['true']==1].copy()

    
lstm_exploit = pd.concat([lstm_exploit,lstm_exploit_1]) 
lstm_exploit

In [ ]:
mnb = MultinomialNB()
mnb.fit(X_train_cf,y_train_cf)
mnb_prediction = mnb.predict(X_test_cf)
y_pred_mnb = mnb.predict_proba(X_test_cf)[:, 1]
y_pred_mnb

lstm_embedded

In [ ]:
evaluation.sort_values(['accuracy', 'loss'])
padded_test_model_seq_reshaped = np.reshape(padded_test_model_seq, (padded_test_model_seq.shape[0], 8000//100, 100, 1))

ms.evaluate(padded_test_model_seq_reshaped, y_test, verbose=1)

In [ ]:
lstm

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Sample data
x = ['W=1', 'W=20', 'W=50', 'W=80']
y1 = [20, 10, 30]
y2 = [15, 25, 20]

# Set the width of the bars
bar_width = 0.35

# Set the positions of the bars on the x-axis
r1 = np.arange(len(x))
r2 = [x + bar_width for x in r1]

# Create the bar plot
plt.bar(r1, y1, color='blue', width=bar_width, edgecolor='white', label='Factor 1')
plt.bar(r2, y2, color='orange', width=bar_width, edgecolor='white', label='Factor 2')

# Add xticks on the middle of the group bars
plt.xticks([r + bar_width/2 for r in range(len(x))], x)

# Add a legend
plt.legend()

# Show the plot
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns

# Sample data

sns.set_palette("pastel")

evaluation = pd.read_csv('lstm_evaluation_metrics.csv')


evaluation.rename(columns={"loss": "Accuracy", "accuracy": "Loss", "w":"Window Size", "l": "Sequence Length"}, inplace=True)

# Create a pandas dataframe
#df = pd.DataFrame(data)

# Create a bar plot using seaborn
#sns.barplot(x='Sequence Length', y='Accuracy', hue='Window Size', data=evaluation)

sns.barplot(x='Sequence Length', y='Loss', hue='Window Size', data=evaluation, dodge=True)


# Show the plot
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns

sns.set_palette("pastel")

evaluation = pd.read_csv('lstm_evaluation_metrics.csv')


evaluation.rename(columns={"loss": "Accuracy", "accuracy": "Loss", "w":"Window Size", "l": "Sequence Length"}, inplace=True)


# Create a bar plot using seaborn
sns.catplot(x='Sequence Length', y='Loss', hue='Window Size', data=evaluation, kind='bar', height=5, aspect=2)

# Show the plot
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns

sns.set_palette("pastel")

evaluation = pd.read_csv('lstm_evaluation_metrics.csv')


evaluation.rename(columns={"loss": "Accuracy", "accuracy": "Loss", "w":"Window Size", "l": "Sequence Length"}, inplace=True)


# Create a bar plot using seaborn
sns.catplot(x='Sequence Length', y='Accuracy', hue='Window Size', data=evaluation, kind='bar', height=5, aspect=2)

# Show the plot
plt.show()